You are an expert Python developer specializing in the Databricks environment. Your task is to create a complete Python script to be executed within a Databricks notebook. The script must perform the following operations:
1.	Data Retrieval from SpaceX API:
o	Interact with the SpaceX v3 REST API (https://api.spacexdata.com/v3).
o	Retrieve data from two specific endpoints: 
	All launches: https://api.spacexdata.com/v3/launches
	All rockets: https://api.spacexdata.com/v3/rockets
o	Handle potential errors during the API calls (e.g., timeouts, non-200 status codes).
2.	Merge Operation:
o	Perform a "merge" (or join) operation between the launches data and the rockets data.
o	Merge Logic: For each launch record, add the rocket's name (rocket_name) from the rockets dataset. The match should be based on the rocket.rocket_id field present in each launch record corresponding to the rocket_id field in each rocket record.
o	The final result should be a list of dictionaries, where each dictionary represents a launch enriched with its corresponding rocket name.
3.	Control Parameters and Debugging:
o	Include variables at the beginning of the script to define the API endpoint URLs, making them easily modifiable.
o	Use Python's standard logging module to provide informative output during execution. Configure logging to display messages at the INFO level.
o	Log key messages such as: starting data retrieval, number of launches retrieved, number of rockets retrieved, starting merge operation, number of records after merge, starting upload to httpbin, upload outcome.
4.	Execution Time Measurement:
o	Code Execution Time: Measure the time taken to perform the main operations (data retrieval + merge). Print this time after the merge operation is complete.
o	Pipeline Execution Time: Measure the total execution time of the entire script (from the beginning until after the upload to httpbin). Print this total time at the end of the script. Use Python's time module.
5.	Upload Result:
o	Take the resulting list from the merge operation.
o	Serialize it into JSON format.
o	Make an HTTP POST request to the https://httpbin.org/post endpoint, sending the resulting JSON data in the request body.
o	Verify the response from httpbin.org (e.g., check the status code) and log the outcome of the upload operation.

In [0]:
# Databricks SpaceX ETL Pipeline
import requests
import json
import time
import logging
from datetime import datetime

# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

# Control Parameters
LAUNCHES_API_URL = "https://api.spacexdata.com/v3/launches"
ROCKETS_API_URL = "https://api.spacexdata.com/v3/rockets"
HTTPBIN_POST_URL = "https://httpbin.org/post"
REQUEST_TIMEOUT = 30  # seconds

def fetch_api_data(url, timeout=REQUEST_TIMEOUT):
    """
    Fetch data from a REST API endpoint with error handling
    """
    try:
        logger.info(f"Fetching data from {url}")
        response = requests.get(url, timeout=timeout)
        response.raise_for_status()  # Raise an exception for non-200 status codes
        return response.json()
    except requests.exceptions.Timeout:
        logger.error(f"Timeout error when fetching data from {url}")
        raise
    except requests.exceptions.HTTPError as http_err:
        logger.error(f"HTTP error occurred: {http_err}")
        raise
    except requests.exceptions.RequestException as req_err:
        logger.error(f"Request error occurred: {req_err}")
        raise
    except json.JSONDecodeError as json_err:
        logger.error(f"JSON decode error: {json_err}")
        raise

def merge_launches_with_rockets(launches, rockets):
    """
    Merge launches data with rockets data based on rocket_id
    """
    logger.info("Starting merge operation between launches and rockets")
    
    # Create a dictionary of rockets with rocket_id as key for faster lookups
    rockets_dict = {rocket["rocket_id"]: rocket for rocket in rockets}
    
    # Merge the data
    merged_launches = []
    for launch in launches:
        launch_data = launch.copy()  # Create a copy to avoid modifying the original
        rocket_id = launch.get("rocket", {}).get("rocket_id")
        
        if rocket_id and rocket_id in rockets_dict:
            launch_data["rocket_name"] = rockets_dict[rocket_id]["rocket_name"]
        else:
            launch_data["rocket_name"] = "Unknown"
            
        merged_launches.append(launch_data)
    
    logger.info(f"Merge completed. Total records after merge: {len(merged_launches)}")
    return merged_launches

def upload_to_httpbin(data):
    """
    Upload data to httpbin.org/post and verify the response
    """
    logger.info(f"Starting upload to {HTTPBIN_POST_URL}")
    try:
        # Convert data to JSON
        json_data = json.dumps(data)
        
        # Upload to httpbin
        response = requests.post(HTTPBIN_POST_URL, data=json_data, 
                                headers={"Content-Type": "application/json"},
                                timeout=REQUEST_TIMEOUT)
        
        response.raise_for_status()
        
        # Verify response
        response_data = response.json()
        if response_data and "data" in response_data:
            logger.info("Upload successful")
            return True
        else:
            logger.warning("Upload completed but response verification failed")
            return False
    except Exception as e:
        logger.error(f"Error uploading data to httpbin: {str(e)}")
        return False

def main():
    """
    Main function to orchestrate the ETL pipeline
    """
    # Start timing the entire pipeline
    pipeline_start_time = time.time()
    
    try:
        # Start timing the data retrieval and merge operations
        operation_start_time = time.time()
        
        # Fetch data from SpaceX API
        launches = fetch_api_data(LAUNCHES_API_URL)
        logger.info(f"Retrieved {len(launches)} launches")
        
        rockets = fetch_api_data(ROCKETS_API_URL)
        logger.info(f"Retrieved {len(rockets)} rockets")
        
        # Merge data
        merged_data = merge_launches_with_rockets(launches, rockets)
        
        # Calculate and log the operation execution time
        operation_duration = time.time() - operation_start_time
        logger.info(f"Data retrieval and merge operations completed in {operation_duration:.2f} seconds")
        
        # Upload result to httpbin
        upload_success = upload_to_httpbin(merged_data)
        upload_status = "Success" if upload_success else "Failed"
        logger.info(f"Data upload status: {upload_status}")
        
        # Calculate and log the total pipeline execution time
        pipeline_duration = time.time() - pipeline_start_time
        logger.info(f"Total pipeline execution time: {pipeline_duration:.2f} seconds")
        
        # Display execution times in a formatted way
        print("\n--- Execution Time Summary ---")
        print(f"Data Retrieval + Merge: {operation_duration:.2f} seconds")
        print(f"Total Pipeline Execution: {pipeline_duration:.2f} seconds")
        print(f"Upload Status: {upload_status}")
        print(f"Total Records Processed: {len(merged_data)}")
        
    except Exception as e:
        logger.error(f"Pipeline execution failed: {str(e)}")
        # Calculate execution time even if there was an error
        pipeline_duration = time.time() - pipeline_start_time
        logger.info(f"Pipeline terminated after {pipeline_duration:.2f} seconds due to an error")

# Entry point for Databricks notebook execution
if __name__ == "__main__":
    logger.info("Starting SpaceX ETL pipeline")
    main()
    logger.info("Pipeline execution completed")